In [1]:
import torch
import torch.nn as nn   
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import time
import random
import pandas as pd
import torch.nn.functional as F
device=torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
cpac_root='/media/D/yazid/ASD-classification-ANEGCN/ABIDEI_CPAC/'
smri_root='/media/D/yazid/ASD-classification-ANEGCN/ABIDEI_sMRI/'
nan_subid=np.load('nan_subid.npy').tolist()
import warnings 
warnings.filterwarnings("ignore")
random_seed = 7777
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [2]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention,self).__init__()
        self.conv1=nn.Conv1d(in_channels=3,out_channels=3,kernel_size=1,padding=0)
        self.conv2=nn.Conv1d(in_channels=116,out_channels=116,kernel_size=1,padding=0)
        self.softmax=nn.Softmax(dim=-1)
    def forward(self,Z,X):
        K=self.conv1(X.permute(0,2,1))
        Q=K.permute(0,2,1)
        V=self.conv2(Z.permute(0,2,1))
        attention=self.softmax(torch.matmul(Q,K))
        out=torch.bmm(attention,V).permute(0,2,1)
        return out
class NEGAN(nn.Module):
    def __init__(self,layer,dropout_rate):
        super(NEGAN,self).__init__()
        self.layer =layer
        self.relu  =nn.ReLU()
        self.atten =nn.ModuleList([Attention() for i in range(layer)])
        self.norm_n=nn.ModuleList([nn.BatchNorm1d(116) for i in range(layer)])
        self.norm_e=nn.ModuleList([nn.BatchNorm1d(116) for i in range(layer)])
        self.node_w=nn.ParameterList([nn.Parameter(torch.randn((3,3),dtype=torch.float32)) for i in range(layer)])
        self.edge_w=nn.ParameterList([nn.Parameter(torch.randn((116,116),dtype=torch.float32)) for i in range(layer)])
        self.line_n=nn.ModuleList([nn.Sequential(nn.Linear(116*3,128),nn.ReLU(),nn.BatchNorm1d(128)) for i in range(layer+1)])
        self.line_e=nn.ModuleList([nn.Sequential(nn.Linear(116*116,128*3),nn.ReLU(),nn.BatchNorm1d(128*3)) for i in range(layer+1)])
        self.clase =nn.Sequential(nn.Linear(128*4*(self.layer+1),1024),nn.Dropout(dropout_rate),nn.ReLU(),
                                   nn.Linear(1024,2))
        self.ones=nn.Parameter(torch.ones((116),dtype=torch.float32),requires_grad=False)
        self._initialize_weights()

    # params initialization
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d,nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    def normalized(self,Z):
        n=Z.size()[0]
        A=Z[0,:,:]
        A=A+torch.diag(self.ones)
        d=A.sum(1)
        D=torch.diag(torch.pow(d,-1))
        A=D.mm(A).reshape(1,116,116)
        for i in range(1,n):
            A1=Z[i,:,:]+torch.diag(self.ones)
            d=A1.sum(1)
            D=torch.diag(torch.pow(d,-1))
            A1=D.mm(A1).reshape(1,116,116)
            A=torch.cat((A,A1),0)
        return A
        
    def update_A(self,Z):
        n=Z.size()[0]
        A=Z[0,:,:]
        Value,_=torch.topk(torch.abs(A.view(-1)),int(116*116*0.2))
        A=(torch.abs(A)>=Value[-1])+torch.tensor(0,dtype=torch.float32)
        A=A.reshape(1,116,116)
        for i in range(1,n):
            A2=Z[i,:,:]
            Value,_=torch.topk(torch.abs(A2.view(-1)),int(116*116*0.2))
            A2=(torch.abs(A2)>=Value[-1])+torch.tensor(0,dtype=torch.float32)
            A2=A2.reshape(1,116,116)
            A=torch.cat((A,A2),0)
        return A
        
    def forward(self,X,Z):
        n=X.size()[0]
        XX=self.line_n[0](X.view(n,-1))
        ZZ=self.line_e[0](Z.view(n,-1))
        for i in range(self.layer):
            A=self.atten[i](Z,X)
            Z1=torch.matmul(A,Z)
            Z2=torch.matmul(Z1,self.edge_w[i])
            Z=self.relu(self.norm_e[i](Z2))+Z
            ZZ=torch.cat((ZZ,self.line_e[i+1](Z.view(n,-1))),dim=1)
            X1=torch.matmul(A,X)
            X1=torch.matmul(X1,self.node_w[i])
            X=self.relu(self.norm_n[i](X1))+X
            #X.register_hook(grad_X_hook)
            #feat_X_hook(X)
            XX=torch.cat((XX,self.line_n[i+1](X.view(n,-1))),dim=1)
        XZ=torch.cat((XX,ZZ),1)
        y=self.clase(XZ)
        #print(self.clase[0].weight)
        return y
def grad_X_hook(grad):
    X_grad.append(grad)
def feat_X_hook(X):
    X_feat.append(X.detach())
X_grad=list()
X_feat=list()

In [3]:
class LabelSmoothLoss(nn.Module):
    
    def __init__(self, smoothing=0.0):
        super(LabelSmoothLoss, self).__init__()
        self.smoothing = smoothing
    
    def forward(self, input, target):
        log_prob = F.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * \
            self.smoothing / (input.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss
def data_split(full_list, ratio, shuffle=True):
    """
    数据集拆分: 将列表full_list按比例ratio（随机）划分为2个子列表sublist_1与sublist_2
    :param full_list: 数据列表
    :param ratio:     子列表1
    :param shuffle:   子列表2
    :return:
    """
    n_total = len(full_list)
    offset = int(n_total * ratio)
    if n_total == 0 or offset < 1:
        return [], full_list
    if shuffle:
        random.shuffle(full_list)
    sublist_1 = full_list[:offset]
    sublist_2 = full_list[offset:]
    return sublist_1, sublist_2
def data_2_k(full_list,k,shuffle=True):
    n_total=len(full_list)
    if shuffle:
        random.shuffle(full_list)
    data_list_list=[]
    for i in range(k):
        data_list_list.append(full_list[int(i*n_total/k):int((i+1)*n_total/k)])
    return data_list_list
def test(device,model,testloader):
    model.eval()
    TP_test,TN_test,FP_test,FN_test=0,0,0,0
    with torch.no_grad():
        for (X,Z,label,sub_id) in testloader:
            TP,TN,FN,FP=0,0,0,0
            n=X.size()[0]
            X=X.to(device)
            Z=Z.to(device)
            label=label.to(device)
            y=model(X,Z)
            _,predict=torch.max(y,1)
            TP+=((predict==1)&(label==1)).sum().item()
            TN+=((predict==0)&(label==0)).sum().item()
            FN+=((predict==0)&(label==1)).sum().item()
            FP+=((predict==1)&(label==0)).sum().item()
            TP_test+=TP
            TN_test+=TN
            FP_test+=FP
            FN_test+=FN
        acc,f1=cal_evaluate(TP_test,TN_test,FP_test,FN_test)
        global max_acc
        global modelname
        global savedModel
        if acc>=max_acc:
            max_acc=acc
            if saveModel:
                torch.save(model.state_dict(),modelname)
                ##read
                #model=NERESGCN(layer)
                #model.load_state_dict(torch.load(PATH))
            #print('Saved the model')
            #print('TEST:  ACC:%.4f  F1:%.4f  [TP:%3d|TN:%3d|FP:%3d|FN:%3d]'%(acc,f1,TP_test,TN_test,FP_test,FN_test)) 
        return acc,f1,TP_test,TN_test,FP_test,FN_test
#计算边节点的字典
def gradient(device,model,dataloader):
    model.eval()
    for (X,Z,A,label,sub_id) in dataloader:
        X=torch.autograd.Variable(X,requires_grad=True)
        x=X.to(device)
        Z=torch.autograd.Variable(Z,requires_grad=True)
        z=Z.to(device)
        A=torch.autograd.Variable(A,requires_grad=True)
        a=A.to(device)
        y=model(x,z,a)
        if (label==torch.FloatTensor([0])).item():
            print('0')
            #y.autograd.backward(torch.FloatTensor([[1.,0.]]).to(device))
            torch.autograd.backward(y,torch.FloatTensor([[1.,0.]]).to(device))
        else:
            print('1')
            torch.autograd.backward(y,torch.FloatTensor([[0.,1.]]).to(device))
        grad_X=X.grad
        grad_Z=Z.grad
        #print(grad_X)
        value_x,index_x=torch.topk(torch.abs(grad_X.view(-1)),10)
        grad_X_topk=(torch.abs(grad_X)>=value_x[-1])
        value_z,index_z=torch.topk(torch.abs(grad_Z.view(-1)),100)
        grad_Z_topk=(torch.abs(grad_Z)>=value_z[-1])
        global gradsave_dict
        if label==torch.FloatTensor([0]).item():
            np.save(gradsave_dict+'/TDC/Z/'+str(sub_id.item()),grad_Z.numpy())
            np.save(gradsave_dict+'/TDC/X/'+str(sub_id.item()),grad_X.numpy())
        else:
            np.save(gradsave_dict+'/ASD/Z/'+str(sub_id.item()),grad_Z.numpy())
            np.save(gradsave_dict+'/ASD/X/'+str(sub_id.item()),grad_X.numpy())
            
def cal_dict():
    index=0
    A={}
    for i in range(116):
        for j in range(i+1,116):
            A[index]=(i,j)
            A[(i,j)]=index
            index+=1
    return A
def cal_evaluate(TP,TN,FP,FN):
    if TP>0:
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        F1 = 2 * r * p / (r + p)
    else:
        F1=0
    acc = (TP + TN) / (TP + TN + FP + FN)
    #print('ACC:%.4f  F1:%.4f  [TP:%d|TN:%d|FP:%d|FN:%d]'%(acc,F1,TP,TN,FP,FN))
    return acc,F1
def data_arange(sites,fmri_root,smri_root,nan_subid):
    asd,tdc=[],[]
    for site in sites:
        mri_asd=os.listdir(smri_root+site+'/group1')
        mri_tdc=os.listdir(smri_root+site+'/group2')
        fmri_asd=os.listdir(fmri_root+site+'/group1_FC')
        fmri_tdc=os.listdir(fmri_root+site+'/group2_FC')
        site_asd=[i for i in mri_asd if i in fmri_asd ]
        site_tdc=[i for i in mri_tdc if i in fmri_tdc ]
        site_asd=[i for i in site_asd if int(i[:5]) not in nan_subid]
        site_tdc=[i for i in site_tdc if int(i[:5]) not in nan_subid]
        asd.append(site_asd)
        tdc.append(site_tdc)
    return asd,tdc
class dataset(Dataset):
    def __init__(self,fmri_root,smri_root,site,ASD,TDC,edge_dict=cal_dict(),topk=True,rate=0.2):
        super(dataset,self).__init__()
        self.fmri=fmri_root
        self.smri=smri_root
        self.ASD=[j for i in ASD for j in i]
        self.TDC=[j for i in TDC for j in i]
        self.data=self.ASD+self.TDC
        random.shuffle(self.data)
        self.data_site={}
        for i in range(len(site)):
            data=ASD[i]+TDC[i]
            for j in data:
                if j not in self.data_site:
                    self.data_site[j]=site[i]                
        self.dict=edge_dict
        self.rate=rate
        self.topk=topk
    def normalize(self,A):
        d=A.sum(1)
        D=torch.diag(torch.pow(d,-1))
        return D.mm(A)
    def __getitem__(self,index):
        data=self.data[index]
        sub_id=int(data[0:5])
        if data in self.ASD:
            data_slow5 =np.load(self.fmri+self.data_site[data]+'/group1_slow5/'+data,allow_pickle=True)
            data_slow4 =np.load(self.fmri+self.data_site[data]+'/group1_slow4/'+data,allow_pickle=True)
            data_voxel =np.load(self.smri+self.data_site[data]+'/group1/'+data,allow_pickle=True)
            data_FCz   =np.load(self.fmri+self.data_site[data]+'/group1_FC/'+data,allow_pickle=True)
        elif data in self.TDC:
            data_slow5 =np.load(self.fmri+self.data_site[data]+'/group2_slow5/'+data,allow_pickle=True)
            data_slow4 =np.load(self.fmri+self.data_site[data]+'/group2_slow4/'+data,allow_pickle=True)
            data_voxel =np.load(self.smri+self.data_site[data]+'/group2/'+data,allow_pickle=True)
            data_FCz   =np.load(self.fmri+self.data_site[data]+'/group2_FC/'+data,allow_pickle=True)
        else:
            print('wrong input')
        data_slow5=(data_slow5-np.min(data_slow5))/(np.max(data_slow5)-np.min(data_slow5))
        data_slow4=(data_slow4-np.min(data_slow4))/(np.max(data_slow4)-np.min(data_slow4))
        if np.any(np.isnan(data_slow5)) or np.any(np.isnan(data_slow4)) or np.any(np.isnan(data_FCz)):
            print('data wronmg')
        #data_FCz=(data_FCz-np.min(data_FCz))/(np.max(data_FCz)-np.min(data_FCz))
        if self.data[index] in self.ASD:
            label=torch.tensor(1)
        else:
            label=torch.tensor(0)
        X=np.zeros((116,3),dtype=np.float32)
        X[:,0]=data_slow5
        X[:,1]=data_slow4
        X[:,2]=data_voxel
        data_FCz=data_FCz.astype(np.float32)
        Z=torch.from_numpy(data_FCz)
        X=torch.from_numpy(X)
        return X,Z,label,sub_id
    def __len__(self):
        return len(self.data)
def get_acc(acc_list,toprate):
    acc_list.sort()
    return acc_list[-int(toprate*len(acc_list))]
def plot_acc(acc_list):
    num_bins=50
    fig,ax=plt.subplots(2)
    n,bins,patches=ax[0].hist(acc_list,num_bins,density=True)
    ax[1].plot(acc_list)
    ax[1].set_ylim(0.4,1)
    plt.show()
    print('Top:10%:',get_acc(acc_list,0.1))
    print('Top:20%:',get_acc(acc_list,0.2))
    print('Max:    ',max(acc_list))
def train_fgsm(model,trainloader,testloader,epsilon):
    result=pd.DataFrame(columns=('Loss','Acc','F1','TP','TN','FP','FN'))
    criterian1=LabelSmoothLoss(0.1).to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
    scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=gmma)
    acc=0.5000
    loss_sum=0
    for j in range(epoch):
        print('\r[%3d/%3d] Loss: %.2f  Acc:%.4f' %(j+1,epoch,loss_sum,acc),end='')
        loss_sum=0
        TP,TN,FP,FN=0,0,0,0
        for (X,Z,A,label,sub_id) in trainloader:
            model.train()
            x=X.to(device)
            z=Z.to(device)
            x.requires_grad=True
            z.requires_grad=True
            label=label.to(device)
            y=model(x,z)
            loss=criterian1(y,label)
            model.zero_grad()
            loss.backward(retain_graph=True)
            sign_grad_x=torch.sign(x.grad.data)
            sign_grad_z=torch.sign(z.grad.data)
            perturbed_x=x+epsilon*sign_grad_x 
            perturbed_z=z+epsilon*sign_grad_z 
            perturbed_x=torch.clamp(perturbed_x,0,1)
            perturbed_z=torch.clamp(perturbed_z,-1,1)
            y=model(perturbed_x,perturbed_z)
            L2=torch.tensor(0,dtype=torch.float32).to(device)
            if L2_lamda>0:
                for name,parameters in model.named_parameters():
                    if name[0:5]=='clase' and  name[-8:]=='0.weight':
                        L2+=L2_lamda*torch.norm(parameters,2)
            loss=0.5*(criterian1(y,label)+loss)+L2
            loss_sum+=loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        acc,f1,TP_test,TN_test,FP_test,FN_test=test(device,model,testloader)
        result.loc[j]={'Loss':loss_sum,'Acc':acc,'F1':f1,'TP':TP_test,'TN':TN_test,'FP':FP_test,'FN':FN_test}
    result.sort_values('Acc',inplace=True,ascending=False)
    print('\n')
    return result.iloc[9]
def train(model,trainloader,testloader):
    result=pd.DataFrame(columns=('Loss','Acc','F1','TP','TN','FP','FN'))
    criterian1=LabelSmoothLoss(0.1).to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
    #optimizer = AdaBelief(model.parameters(), lr=1e-4, eps=1e-8, betas=(0.9,0.999), weight_decay=L2_lamda,weight_decouple = True, rectify = False)
    scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=gmma)
    acc=0.5000
    loss_sum=0
    for _ in range(epoch):
        print('\rLoss: {:.2f}  Acc:{:.4f}'.format(loss_sum,acc),end='')
        loss_sum=0
        TP,TN,FP,FN=0,0,0,0
        time_start=time.time()
        for (X,Z,A,label,sub_id) in trainloader:
            #print(A)
            #print(Z)
            #print(X.shape,torch.mean(X),torch.std(X))
            #X=X+torch.randn(X.shape)*X.std(0)
            #Z=Z+torch.randn(Z.shape)*Z.std(0)
            model.train()
            X=X.to(device)
            Z=Z.to(device)
            label=label.to(device)
            y=model(X,Z)
            #print(y)
            _,predict=torch.max(y,1)
            TP+=((predict==1)&(label==1)).sum().item()
            TN+=((predict==0)&(label==0)).sum().item()
            FN+=((predict==0)&(label==1)).sum().item()
            FP+=((predict==1)&(label==0)).sum().item()
            loss=criterian1(y,label)
            L1=torch.tensor(0,dtype=torch.float32).to(device)
            L2=torch.tensor(0,dtype=torch.float32).to(device)
            #print(model.parameters.weit_n)
            if L1_lamda>0 or L2_lamda>0:
                for name,parameters in model.named_parameters():
                    if name[0:5]=='clase' and  name[-8:]=='0.weight':
                        L1+=L1_lamda*torch.norm(parameters,1)
                        L2+=L2_lamda*torch.norm(parameters,2)
            loss+=(L1+L2)
            loss_sum+=loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
        time_end=time.time()
        time_cost=(time_end-time_start)/60.0
        acc,f1,TP_test,TN_test,FP_test,FN_test=test(device,model,testloader)
        result.loc[j]={'Loss':loss_sum,'Acc':acc,'F1':f1,'TP':TP_test,'TN':TN_test,'FP':FP_test,'FN':FN_test}
        #acc,f1=cal_evaluate(TP,TN,FP,FN)
        #print("[%2d/%d] ACC:%.2f F1:%.2f Loss: %.4f [TP:%3d|TN:%3d|FP:%3d|FN:%3d] CostTime:%4.1f min | RestTime:%.2f h" %(
         #   j+1,epoch,acc,f1,loss_sum,TP,TN,FP,FN,time_cost,time_cost/60*(epoch-1-j)))
    #losses.append(loss_sum)
    #print(model.parameters())
        #acc,f1=test(device,model,testloader)
        #print(acc)
    #plot_acc(result['Acc'],result['Loss'])
    result.sort_values('Acc',inplace=True,ascending=False)
    print('\n')
    return result.iloc[9] 
def train_pgd(model,trainloader,testloader,eps=0.05,iters=10,alpha=2/255):
    result=pd.DataFrame(columns=('Loss','Acc','F1','TP','TN','FP','FN'))
    criterian1=LabelSmoothLoss(0.1).to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
    for j in range(epoch):
        loss_sum=0.
        TP,TN,FP,FN=0,0,0,0
        model.train()
        for (X,Z,label,sub_id) in trainloader:
            model.train()
            x=X.to(device)
            z=Z.to(device)
            label=label.to(device)
            pretu_x,pretu_z=x,z
            ori_x,ori_z=x.data,z.data
            for i in range(iters):
                pretu_x.requires_grad=True
                pretu_z.requires_grad=True
                y=model(pretu_x,pretu_z)
                loss=criterian1(y,label)
                model.zero_grad()
                loss.backward()
                adv_x=pretu_x+alpha*torch.sign(pretu_x.grad.data)
                adv_z=pretu_z+alpha*torch.sign(pretu_z.grad.data)
                eta_x=torch.clamp(adv_x-ori_x,min=-eps,max=eps)
                eta_z=torch.clamp(adv_z-ori_z,min=-eps,max=eps)
                pretu_x=torch.clamp(ori_x+eta_x,min=0,max=1).detach_()
                pretu_z=torch.clamp(ori_z+eta_z,min=-1,max=1).detach_()
            y=model(x,z)
            yy=model(pretu_x,pretu_z)
            L2=torch.tensor(0,dtype=torch.float32).to(device)
            if L2_lamda>0:
                for name,parameters in model.named_parameters():
                    if name[0:5]=='clase' and  name[-8:]=='0.weight':
                        L2+=L2_lamda*torch.norm(parameters,2)
            loss=0.5*(criterian1(yy,label)+criterian1(y,label))+L2
            loss_sum+=loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (j+1)%10==0:
            acc,f1,TP_test,TN_test,FP_test,FN_test=test(device,model,testloader)
            result.loc[(j+1)//10]=[loss_sum,acc,f1,TP_test,TN_test,FP_test,FN_test]
    result.sort_values('Acc',inplace=True,ascending=False)
    print(' FinalAcc: {:.4f}'.format(result.iloc[0]['Acc']))
    return result.iloc[0]['Acc']

In [4]:
# leave one site out
setup_seed(random_seed)
L1_lamda=0.0
L2_lamda=0.0001
learning_rate=0.0001
epoch   =100
batch_size=64
gmma    =0.95
layer   =5
all_site=os.listdir(cpac_root)
site_count={'CALTECH':19,'CMU':13,'KKI':51,'LEUVEN':62,'MAXMUN':55,'NYU':181,
           'OHSU':28,'OLIN':36,'PITT':40,'SBL':9,'SDSU':27,'STANFORD':40,
           'TRINITY':48,'UCLA':99,'UM':144,'USM':100,'YALE':55}
for epision in [0.001,0.005,0.01,0.02]:
    sum_acc=0
    for site in all_site:
        test_site =[i for i in all_site if i==site]
        train_site=[i for i in all_site if i!=site]
        global max_acc
        max_acc=0.5
        global modelname
        modelname='/media/dm/0001A094000BF891/Yazid/SAVEDModels/adversialtrained/models_{}'.format(test_site[0])
        global saveModel
        saveModel=False
        train_asd,train_tdc=data_arange(train_site,fmri_root=cpac_root,smri_root=smri_root,nan_subid=nan_subid)
        test_asd,test_tdc  =data_arange(test_site, fmri_root=cpac_root,smri_root=smri_root,nan_subid=nan_subid)
        trainset=dataset(site=train_site,fmri_root=cpac_root,smri_root=smri_root,ASD=train_asd,TDC=train_tdc,edge_dict=cal_dict())
        trainloader=DataLoader(trainset,batch_size=batch_size,shuffle=True,drop_last=True)
        testset=dataset(site=test_site,fmri_root=cpac_root,smri_root=smri_root,ASD=test_asd,TDC=test_tdc,edge_dict=cal_dict())
        testloader=DataLoader(testset,batch_size=1)
        model=NEResGCN(layer,dropout_rate=0.2).to(device)
        acc=train_pgd(model,trainloader,testloader,eps=epision,iters=10,alpha=epision/5)
        sum_acc+=(site_count[test_site[0]]*acc)
    print('dropout_rate:%.4f   Acc:%.4f'%(epision,sum_acc/1007))

 FinalAcc: 0.6875
 FinalAcc: 0.6774
 FinalAcc: 0.7017
 FinalAcc: 0.7778
 FinalAcc: 0.8462
 FinalAcc: 0.7900
 FinalAcc: 0.6667
 FinalAcc: 0.6182
 FinalAcc: 0.6078
 FinalAcc: 0.6389
 FinalAcc: 0.7708
 FinalAcc: 0.7000
 FinalAcc: 0.6909
 FinalAcc: 0.7500
 FinalAcc: 0.7172
 FinalAcc: 0.7895
 FinalAcc: 0.6000
dropout_rate:0.0010   Acc:0.7001
 FinalAcc: 0.6806
 FinalAcc: 0.6452
 FinalAcc: 0.7127
 FinalAcc: 0.8889
 FinalAcc: 0.8462
 FinalAcc: 0.7700
 FinalAcc: 0.6296
 FinalAcc: 0.6727
 FinalAcc: 0.6667
 FinalAcc: 0.6944
 FinalAcc: 0.6667
 FinalAcc: 0.7500
 FinalAcc: 0.7455
 FinalAcc: 0.6786
 FinalAcc: 0.7172
 FinalAcc: 0.8421
 FinalAcc: 0.6250
dropout_rate:0.0050   Acc:0.7051
 FinalAcc: 0.7292
 FinalAcc: 0.6774
 FinalAcc: 0.7017
 FinalAcc: 0.7778
 FinalAcc: 0.8462
 FinalAcc: 0.8000
 FinalAcc: 0.8148
 FinalAcc: 0.6364
 FinalAcc: 0.7059
 FinalAcc: 0.7222
 FinalAcc: 0.7292
 FinalAcc: 0.8000
 FinalAcc: 0.7818
 FinalAcc: 0.6786
 FinalAcc: 0.7374
 FinalAcc: 0.8421
 FinalAcc: 0.6000
dropout_rate:0.0

**Epsilon**:***0.0010*** **Acc**: ***703***<br>
**Epsilon**:***0.0050*** **Acc**: ***715***<br>
**Epsilon**:***0.0100*** **Acc**: ***715***<br>
**Epsilon**:***0.0200*** **Acc**: ***727***<br>
**Epsilon**:***0.0500*** **Acc**: ***717***<br>
**Epsilon**:***0.1000*** **Acc**: ***675***<br>
**Epsilon**:***0.2000*** **Acc**: ***627***<br>